In [2]:
import pandas as pd
import json
from helpers.utils import normalize_name

In [11]:
with open('utils/name_map_odds.json', 'r') as f:
    name_map = json.load(f)

In [12]:
odds = pd.read_csv('data/odds_first_basket.csv')[['name']].drop_duplicates().sort_values('name').reset_index(drop = True)
metadata = pd.read_csv('data/player_metadata.csv')
odds['name_norm'] = odds['name'].apply(normalize_name)
assert odds['name_norm'].value_counts().max() == 1, 'Normalized name duplicate in odds'
odds['player_id'] = odds['name'].map(name_map)
assert odds['player_id'].value_counts().max() == 1, 'Dupplicate player_id on odds'
# Subset of players needing to be mapped
odds_ = odds.copy()[odds['player_id'].isna()]

In [14]:
merged = pd.merge(
    odds_[['name', 'name_norm']],
    metadata[['name_norm', 'player_id']],
    on = 'name_norm',
    how = 'left'
    )

merged_check = merged.copy()[merged['player_id'].notna()]
assert odds['name_norm'].value_counts().max() == 1, 'Normalized name duplicate in merged_check'
assert odds['player_id'].value_counts().max() == 1, 'Duplicated player_id in merged_check'

name_map_add = dict(zip(merged_check['name'], merged_check['player_id']))

In [ ]:
with open('utils/name_map_odds.json', 'w') as f:
    json.dump(name_map | name_map_add, f, indent = 4)

In [18]:
merged_need = merged.copy()[merged['player_id'].isna()]

In [19]:
merged_need

,name,name_norm,player_id


In [20]:
players_need_id = merged_need['name'].to_list()

In [21]:
players_need_id

[]